In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, LSTM, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape, ZeroPadding1D, Cropping1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15424352845930569892
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9718310882
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8613941259193930723
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5"
]


### Load Data

In [2]:
test_data_domain = None
test_data_phase = None

with open('dataset/test_samples.pickle', 'rb') as data:
    test_samples = pickle.load(data)

with open('generatedData/3/generatedData.pickle', 'rb') as data:
    generated_data = pickle.load(data)

with open('dataset/test_set_differential/test_set_fdi.pickle', 'rb') as data:
    test_data_fdi = pickle.load(data)
    
with open('dataset/test_set_differential/test_set_tap_setting_hard.pickle', 'rb') as data:
    test_data_tap_setting = pickle.load(data)
    
with open('dataset/test_set_differential/test_set_replay_hard.pickle', 'rb') as data:
    test_data_replay = pickle.load(data)

### Data Preprocessing

In [3]:
# data separation according to window size
sequenceLen = 48 # in training
testSequenceLen = test_samples.shape[1]
testSamplesSize = test_samples.shape[0]
dimensionsCount = 6

testSetSize = testSamplesSize + generated_data.shape[0]
numberOfAttackSamples = generated_data.shape[0]
attackSamplesStartIndex = testSetSize - numberOfAttackSamples

test_data = np.concatenate((test_samples, generated_data));

In [4]:
test_data.shape

(14148, 95, 6)

### Test set preparation

In [5]:
testSet = list()
for entry in test_data:
    for i in range(48):
        testSet.append(list(entry[i:i+48,:]))
testSet = np.array(testSet)
ConvTestData = np.reshape(testSet, (testSetSize*48,sequenceLen*dimensionsCount,1), order='C')
fullyConnectedTestData = np.reshape(testSet, (testSetSize*48,sequenceLen*dimensionsCount), order='C')
LSTMTestData = np.reshape(testSet, (testSetSize*48,sequenceLen,dimensionsCount), order='C')
    
test_labels = np.zeros(testSetSize)
test_labels[attackSamplesStartIndex:testSetSize] = np.ones(numberOfAttackSamples) 

### Functions
Here we defined rankedPrecision() and rankedRecal() functions which are our evaluation functions in anomaly detection. The overall idea here is that, we first sort the samples loss from highest to lowest. Then, calculate precision and recall in top-K samples using definitions below:<br>
<p style="text-align:center">
$Precision=\frac{True Positive}{True Positive + False Positive}$<br/><br/>
$Recall=\frac{True Positive}{True Positive + False Negative}$<br/><br/>
</p>
As we know the number of anomalies in our test set, we pick K to be that number.

In [6]:
def rankedPrecision(mse_labels):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
    TP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 1)
    FP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 0)
    TN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 0)
    FN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 1)
    precision = TP/(TP+FP)
    return precision
    
    
def rankedRecall(mse_labels):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
    TP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 1)
    FP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 0)
    TN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 0)
    FN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 1)
    recall = TP/(TP+FN)
    return recall

# show_curve() is for plotting training and validation losses
def show_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [7]:
def rankedPrecisionThre(mse_labels,threshold):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    TP = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))
    FP = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] >= threshold))
    TN = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] < threshold))
    FN = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] < threshold))
    precision = TP/(TP+FP)
    return precision
    
    
def rankedRecallThre(mse_labels,threshold):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    TP = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))
    FP = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] >= threshold))
    TN = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] < threshold))
    FN = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] < threshold))
    recall = TP/(TP+FN)
    return recall

### 1-D Convolutional Network Final evaluation

In [9]:
precisions = []
recalls = []
model = load_model('models_differential/conv.h5')

predicted = model.predict(ConvTestData)
mse = (np.square(ConvTestData - predicted)).mean(axis=1)
mse = mse.reshape(testSetSize,48)
mse = mse.mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision = rankedPrecision(mse_label)
recall = rankedRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
print("Precision is: ", precision)
print("Recall is: ", recall)

convNetPrecisions_domain = precisions

Precision is:  1.0
Recall is:  1.0


In [10]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]

totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
print(min(sorted_mse_label[sorted_mse_label[:,1] == 1,0])) # 0.010118610983828593
print(max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])) 

1.4760437932762619e-05
3.0375225336844425e-06


In [11]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
minMSE = min(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
maxMSE = max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
thre = maxMSE + (maxMSE - minMSE)/10
threshold = 1
print((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))

precision = rankedPrecisionThre(mse_label,thre)
recall = rankedRecallThre(mse_label,thre)
print("Precision by threshold is: ", precision)
print("Recall by threshold is: ", recall)

[False False False ... False False False]
Precision by threshold is:  1.0
Recall by threshold is:  1.0


### Auto-encoder Final evaluation

In [12]:
precisions = []
recalls = []
model = load_model('models_differential/autoencoder.h5')

predicted = model.predict(fullyConnectedTestData)
mse = (np.square(fullyConnectedTestData - predicted)).mean(axis=1)
mse = mse.reshape(testSetSize,48)
mse = mse.mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision = rankedPrecision(mse_label)
recall = rankedRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
print("Precision is: ", precision)
print("Recall is: ", precision)

fullyConnectedPrecisions_domain = precisions

Precision is:  1.0
Recall is:  1.0


In [13]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]

totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
print(min(sorted_mse_label[sorted_mse_label[:,1] == 1,0])) # 0.010118610983828593
print(max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])) 

0.0001339250640816447
0.00012368260877431675


In [14]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
minMSE = min(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
maxMSE = max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
thre = maxMSE + (maxMSE - minMSE)/10
threshold = 1
print((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))

precision = rankedPrecisionThre(mse_label,thre)
recall = rankedRecallThre(mse_label,thre)
print("Precision by threshold is: ", precision)
print("Recall by threshold is: ", recall)

[False False False ... False False False]
Precision by threshold is:  1.0
Recall by threshold is:  1.0


### Final LSTM evaluation

In [15]:
precisions = []
recalls = []
model = load_model('models_differential/lstm.h5')
predicted = model.predict(LSTMTestData)
mse = (np.square(LSTMTestData - predicted)).mean(axis=2).mean(axis=1)
mse = mse.reshape(testSetSize,48)
mse = mse.mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision = rankedPrecision(mse_label)
recall = rankedRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
print("Precision is: ", precision)
print("Recall is: ", precision)

LSTMPrecisions_domain = precisions

Precision is:  1.0
Recall is:  1.0


In [16]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]

totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
print(min(sorted_mse_label[sorted_mse_label[:,1] == 1,0])) # 0.010118610983828593
print(max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])) 

6.721195468540186e-06
2.3945688877362906e-07


In [17]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
minMSE = min(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
maxMSE = max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
thre = maxMSE + (maxMSE - minMSE)/10
threshold = 1
print((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))

precision = rankedPrecisionThre(mse_label,thre)
recall = rankedRecallThre(mse_label,thre)
print("Precision by threshold is: ", precision)
print("Recall by threshold is: ", recall)

[False False False ... False False False]
Precision by threshold is:  1.0
Recall by threshold is:  1.0
